It helps crypto traders to save the prices and their conversion rate to other coin/tokens at the moment they buy or convert their assets (Trade class). And shows which conversion is beneficial by using second class - Profit. If select_only set to true it will show only profitable conversions. If select set to true it will want a coin and return data for that specific token/coin. If return_ set to true it will return the whole table

In [1]:
import requests as rs
from bs4 import BeautifulSoup as bs
import pandas as pd
from pandas import DataFrame as dtf
import re
from datetime import datetime as dt
from IPython.display import display as dp

# Trade

In [2]:
class Trade:

  def __init__(self, crypto = '', save = False) -> None:
    self.crypto = crypto.upper() if crypto else 'DOGE'
    self.save = save

  def get_current_prices(self):

    url = 'https://www.binance.com/en/markets?p'

    response = rs.get(url)
    if response.status_code != 200:            return 'Not [200]'

    soup = bs(response.content, 'html.parser')

    self.df = dtf({
        'full_name':      [name.string for name in soup.find_all(class_ = 'css-1ap5wc6')],
        'abb':            [abb.string for abb in soup.find_all(class_ = 'css-1x8dg53')],
        'price':          [price.string for price in soup.find_all(class_ = 'css-ovtrou')]
        })

    try:                  self.df['price_float'] = [float(re.sub('[\$\,]', '', price)) for price in self.df.price]
    except:               return "Can't convert the price into float!"

    try:                  ex_crypto = float(self.df[self.df['abb'] == self.crypto]['price_float'])
    except:               print("The wanted asset [%s] haven't been found!" % self.crypto);   return self.df
                         
    self.df[ self.crypto + '\n' + 
            dt.today().strftime('%d %b %Y') ]   =  [
                 ex_crypto / price for price in self.df['price_float']]



    if self.save:

      try:                self.df.to_csv('/content/drive/MyDrive/trade/%s%s.csv' % (self.crypto.lower(), dt.today().strftime('_%d%m%y')))
      except:             print('An error occured => [save]!')

    return self.df

# Profit

In [3]:
class Profit: 

  def __init__(self, crypto, history, return_ = False, select = False, only_prof = False):
    self.history = history
    self.crypto = crypto.upper()
    self.return_ = return_
    self.select = select
    self.only_prof = only_prof

  def show_profit(self):

    path = self.crypto.lower() + self.history.strftime('_%d%m%y')
    tr_ = (self.crypto, self.history.strftime('%d %b %Y'))
    cr_ = (self.crypto, dt.today().strftime('%d %b %Y'))
    cur = Trade(self.crypto.lower()).get_current_prices()

    if not len(cur):        return 
    print(('/content/drive/MyDrive/trade/%s.csv' % path))
    try:                    trade = pd.read_csv('/content/drive/MyDrive/trade/%s.csv' % path); print('\ttrade retrieved!')
    except:                 print("The file can't be opened. Check if the directory or file name is correct!"); return

    try:                    merged = pd.merge(trade[['full_name', 'abb', 'price', 'price_float', '%s\n%s' % tr_]
                                                    ],cur[['price', 'abb', 'price_float', '%s\n%s' % cr_]
                                                          ], on='abb', suffixes = ['_trade', '_cur']); print('\tfiles merghed!')
    except:                 print("Merging failed!");                                     return
    
    print('%s\n%s\n' % cr_ + '%s\n%s' % tr_)

    try:                    merged['cur_prof'] = merged[
        '%s\n%s_cur' % cr_ if cr_ == tr_ else '%s\n%s' % cr_] - merged[
            '%s\n%s_trade' % tr_ if cr_ == tr_ else '%s\n%s' % tr_]; print('\tCurrent profit found!') 

    except:                 print('Current profit observation failed!');                  return

    try:                    merged['cur_prof_percent'] = merged[
        '%s\n%s_cur' % cr_ if cr_ == tr_ else '%s\n%s' % cr_] / merged[
            '%s\n%s_trade' % tr_ if cr_ == tr_ else '%s\n%s' % tr_] * 100 - 100; print('\tCurrent profit persentage found!')
    except:                 print('Current profit percentage observation failed!');       return

    try:                    merged.drop(['price_float_trade','price_float_cur'], axis=1, inplace=True); print("['price_float_trade','price_float_cur'] dropped!")
    except:                 print("Couldn't drop columuns!");                             return


    if not self.select and not self.only_prof and self.return_:                           return merged[
        
        ['full_name', 'abb', 
         'price_trade', 
         'price_cur', 
         '%s\n%s_trade' % tr_ if cr_ == tr_ else '%s\n%s' % tr_, 
         '%s\n%s_cur' % cr_ if cr_ == tr_ else '%s\n%s' % cr_, 
         'cur_prof', 
         'cur_prof_percent']

    ]

    if not self.select and self.only_prof:                                                return merged[
        
        merged.cur_prof > 0

    ]

    if self.select:         target = input('Asset: ').upper()

    try:                    selection = merged[merged['abb'] == target];                  return selection
    except:                 print("Asset not found!");                                    return

    


# Run


In [ ]:
# Trade

df = Trade(input('Asset: '), save = bool(input('Save? 1/0: '))).get_current_prices()

Asset: busd
Save? 1/0: 1


In [ ]:
# Profit

def profit():
  try:
    return Profit(input('Asset: '),
                  dt(int(input('year: ')), int(input('month: ')), int(input('day: '))),
                  return_=True, select=False, only_prof=False).show_profit()
  except: 
    print('Error occured!')
2
dp(profit())

#Outputs
This part is just to show you the outputs. If you have anyquestion about this project you can find me:

email: farhad.szd@gmail.com

whatsapp: +994 55 736 7002

instagram: farhad_zada_

In [4]:
# Trade

df = Trade(input('Asset: '), save = bool(input('Save? 1/0: '))).get_current_prices()
df

Asset: doge
Save? 1/0: 0


,full_name,abb,price,price_float,DOGE\n03 Jan 2023
0,Bitcoin,BTC,"$16,723.05",1.672305e+04,0.000004
1,Ethereum,ETH,"$1,215.26",1.215260e+03,0.000059
2,TetherUS,USDT,$1.00,1.000000e+00,0.071750
3,BNB,BNB,$246.00,2.460000e+02,0.000292
4,Ripple,XRP,$0.34560000,3.456000e-01,0.207610
5,BUSD,BUSD,$1.00,1.000000e+00,0.071750
6,Dogecoin,DOGE,$0.07175000,7.175000e-02,1.000000
7,Cardano,ADA,$0.25360000,2.536000e-01,0.282926
8,Polygon,MATIC,$0.78160000,7.816000e-01,0.091799
9,Litecoin,LTC,$75.53,7.553000e+01,0.000950


In [5]:
# Profit


# SELECT 

def profit():
  try:
    return Profit('doge',
                  dt(2022, 12, 30),
                  return_=0, select=1, only_prof=0).show_profit()
  except: 
    print('Error occured!')

dp(profit()) 

/content/drive/MyDrive/trade/doge_301222.csv
	trade retrieved!
	files merghed!
DOGE
03 Jan 2023
DOGE
30 Dec 2022
	Current profit found!
	Current profit persentage found!
['price_float_trade','price_float_cur'] dropped!
Asset: busd


,full_name,abb,price_trade,DOGE\n30 Dec 2022,price_cur,DOGE\n03 Jan 2023,cur_prof,cur_prof_percent
5,BUSD,BUSD,$1.00,0.06977,$1.00,0.07175,0.00198,2.837896


In [6]:
# only_prof


# only_prof returns only profitable coins 

def profit():
  try:
    return Profit('doge',
                  dt(2022, 12, 30),
                  return_=0, select=0, only_prof=1).show_profit()
  except: 
    print('Error occured!')

dp(profit()) 

/content/drive/MyDrive/trade/doge_301222.csv
	trade retrieved!
	files merghed!
DOGE
03 Jan 2023
DOGE
30 Dec 2022
	Current profit found!
	Current profit persentage found!
['price_float_trade','price_float_cur'] dropped!


,full_name,abb,price_trade,DOGE\n30 Dec 2022,price_cur,DOGE\n03 Jan 2023,cur_prof,cur_prof_percent
0,Bitcoin,BTC,"$16,552.84",0.000004,"$16,724.20",0.000004,7.460565e-08,1.770009
1,Ethereum,ETH,"$1,194.45",0.000058,"$1,215.65",0.000059,6.018750e-07,1.030399
2,TetherUS,USDT,$1.00,0.069770,$1.00,0.071740,1.970000e-03,2.823563
3,BNB,BNB,$244.80,0.000285,$245.70,0.000292,6.973922e-06,2.446920
4,Ripple,XRP,$0.33920000,0.205690,$0.34540000,0.207701,2.011357e-03,0.977859
5,BUSD,BUSD,$1.00,0.069770,$1.00,0.071740,1.970000e-03,2.823563
8,Polygon,MATIC,$0.76290000,0.091454,$0.78260000,0.091669,2.151327e-04,0.235237
9,TRON,TRX,$0.05387000,1.295155,$0.05532000,1.296819,1.663508e-03,0.128441
12,SHIBA INU,SHIB,$0.00000810,8613.580247,$0.00000816,8791.666667,1.780864e+02,2.067508
16,Wrapped Bitcoin,WBTC,"$16,503.71",0.000004,"$16,720.00",0.000004,6.313556e-08,1.493437


In [7]:
# return_


# full table

def profit():
  try:
    return Profit('doge',
                  dt(2022, 12, 30),
                  return_=1, select=0, only_prof=0).show_profit()
  except: 
    print('Error occured!')

dp(profit()) 

/content/drive/MyDrive/trade/doge_301222.csv
	trade retrieved!
	files merghed!
DOGE
03 Jan 2023
DOGE
30 Dec 2022
	Current profit found!
	Current profit persentage found!
['price_float_trade','price_float_cur'] dropped!


,full_name,abb,price_trade,price_cur,DOGE\n30 Dec 2022,DOGE\n03 Jan 2023,cur_prof,cur_prof_percent
0,Bitcoin,BTC,"$16,552.84","$16,724.20",0.000004,0.000004,7.460565e-08,1.770009
1,Ethereum,ETH,"$1,194.45","$1,215.65",0.000058,0.000059,6.018750e-07,1.030399
2,TetherUS,USDT,$1.00,$1.00,0.069770,0.071740,1.970000e-03,2.823563
3,BNB,BNB,$244.80,$245.70,0.000285,0.000292,6.973922e-06,2.446920
4,Ripple,XRP,$0.33920000,$0.34540000,0.205690,0.207701,2.011357e-03,0.977859
5,BUSD,BUSD,$1.00,$1.00,0.069770,0.071740,1.970000e-03,2.823563
6,Dogecoin,DOGE,$0.06977000,$0.07174000,1.000000,1.000000,0.000000e+00,0.000000
7,Cardano,ADA,$0.24260000,$0.25380000,0.287593,0.282664,-4.929231e-03,-1.713962
8,Polygon,MATIC,$0.76290000,$0.78260000,0.091454,0.091669,2.151327e-04,0.235237
9,TRON,TRX,$0.05387000,$0.05532000,1.295155,1.296819,1.663508e-03,0.128441
